In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly import graph_objects as go

import matplotlib
import plotly
import sklearn
import torch

from torch import nn, optim

from IPython.display import display
plt.style.use("ggplot")

print("package版本信息：")
print("numpy:      ", np.__version__)
print("pandas:     ", pd.__version__)
print("matplotlib: ", matplotlib.__version__)
print("sklearn:    ", sklearn.__version__)
print("seaborn:    ", sns.__version__)
print("plotly:     ", plotly.__version__)
print("PyTorch:     ", torch.__version__)

package版本信息：
numpy:       1.19.2
pandas:      1.0.1
matplotlib:  3.3.1
sklearn:     0.23.2
seaborn:     0.11.1
plotly:      4.14.1
PyTorch:      1.2.0


# Pytorch-Transformer

In [2]:
from torch import nn

## MultiheadAttention Layer

需要注意的是，pytorch源码中 `MultiheadAttention` 被放在了 Non-linear Activations 这一类中（并且是放在`activation.py`文件中的）。

`MultiheadAttention(embed_dim, num_heads, dropout=0.0, bias=True, add_bias_kv=False, add_zero_attn=False, kdim=None, vdim=None)`
+ 参数
  + `embed_dim`，
  + `num_heads`，
  + `bias`，
  + `add_bias_kv`
  + `add_zero_attn`
  + `kdim`，key 中的特征数量
  + `vidm`，value 中的特征数量


+ `forward(query, key, value, key_padding_mask=None, need_weights=True, attn_mask=None)`
  + `query`, `key`, `value`，self-attention的三个输入向量
    + `query.shape` = $(L, N, E)$，
    + `key.shape` = $(S, N, E)$
    + `value.shape` = $(S, N, E)$
  
  其中 $E$ 是输入的 embedding 长度， $S$ 是输入序列的长度，$L$ 是输出序列的长度， $N$ 是batch size.
  + `key_padding_mask`，
  + `need_weights`，bool，表示是否需要输出attention的weights
  + `attn_mask`，2维或者3维的mask


+ `forward()`方法返回值
  + `attn_output`，shape= $(L, N, E)$，$L$ 是最长的序列长度
  + `attn_output_weights`，shape=$(N, L, S)$

In [4]:
embed_dim = 512
num_heads = 8
multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)

In [7]:
E, S, L, N = embed_dim, 20, 10, 5
query = torch.rand(L, N, E)
key = torch.rand(S, N, E)
value = torch.rand(S, N, E)

attn_output, attn_output_weights = multihead_attn(query, key, value)

In [9]:
print(attn_output.shape)
print(attn_output_weights.shape)

torch.Size([10, 5, 512])
torch.Size([5, 10, 20])


## Encoder Layer

`nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu')`
+ 参数
  + `d_model`，输入的特征个数
  + `nhead`，multiheadattention中的head个数
  + `dim_feedforward`，前馈神经网络里的个数
  
+ `.forward(src, src_mask=None, src_key_padding_mask=None)` 方法
  + `src`，输入的sequence
  + `src_mask`，输入sequence的mask
  + `src_key_padding_mask`，每个batch中src的padding
  
+ `forward()`方法返回值


In [ ]:
d_model = 512
nhead = 8
dim_ffn = 
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
src = torch.rand(10, 32, 512)
out = encoder_layer(src)

# 基础

huggingface/transformer


transformer 包里最重要的三个子模块是：
1. [Configuration](https://huggingface.co/transformers/main_classes/configuration.html)  
基础的类是 `PretrainedConfig`（好像也只有这一个类），用于 导出\导入 模型的配置。
2. [Models](https://huggingface.co/transformers/main_classes/model.html)  
基础的类有如下三个，用于构建模型：
  + `PreTrainedModel`，pytorch的模型都继承了这个类，继承了 `torch.nn.Module` 类.
  + `TFPreTrainedModel`，Tensorflow2.0里的模型都继承了这个类，继承了 `tf.keras.Model` 类.
  + `FlaxPreTrainedModel`
3. [Tokenizer](https://huggingface.co/transformers/main_classes/tokenizer.html)  
包含两个基础的类，用于处理文本特征：
  + `PreTrainedTokenizer`，这个比较慢
  + `PreTrainedTokenizerFast`，这个比较快
  
上面这三个基础模块，都有如下两个方法用于导入\导出配置：
+ `.from_pretrained()`，导入预训练的模型
+ `.save_pretrained()`，导出训练好的模型

**第一次使用这些语句的时候，如果本地没有对应的模型，会下载这些模型**。
  
在上面的3个基础子模块之上，还提供了如下几个子模块工具：

4. [Pipepline](https://huggingface.co/transformers/main_classes/pipelines.html)  
用于快速使用模型，是对上面三个类的封装
5. [Trainer](https://huggingface.co/transformers/main_classes/trainer.html)  
用于快速训练或者 fine-tune 模型

transformer源码中，所有的模型结构都存放在 `/src/transformers/models`文件夹下，每个模型对应于一个文件夹（比如BERT对应的就是`bert`文件夹），每个模型的文件夹内，主要有如下几个`.py`文件，其中的 `xxx` 是对应模型的名称：
+ **`configuration_xxx.py`**，编写了该模型对应的`PretrainedConfig`子类，比如BERT就是`BertConfig`。
+ **`modeling_xxx.py`**，存放了pytorch编写的模型结构，要用到的模型类都放在这里面
+ `modeling_tf_xxx.py`，存放tensorflow编写的模型结构
+ `modeling_flax_xxx.py`
+ **`tokenization_xxx.py`**，用于 分词的 实现类，比如BERT就是`BertTokenizer`类
+ `tokenization_xxx_fast.py`，快速分词的实现类
+ `convert_*.py`，用于将其它配置文件转换成对应的模型。


我一般常用的两个模型是 **BERT** 和 **Distil-BERT**，所以下面会着重关注这两个模型里的实现。

+ [BERT Models](https://huggingface.co/transformers/model_doc/bert.html) 包含如下类（以pytorch为例）
  + `BertConfig`，配置类
  + `BertTokenizer`，分词的类，基于 `WordPiece`
  + `BertModel`，最基本的 BERT 模型类，它返回的是 BERT 模型的原始结果，包括隐藏层状态.
  + `BertForPreTraining`，在 `BertModel`的输出上加了一层处理，以下的几个模型都是如此.
  + `BertForMaskedLM`
  + `BertForNextSentencePrediction`
  + `BertForSequenceClassification`
  + 还有其他的一些基础工具类和其他任务对于的BERT模型类，这里就不介绍了。
  

+ [DistilBERT Models](https://huggingface.co/transformers/model_doc/distilbert.html) 包含如下类

In [ ]:
from transformers import Ber